In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt #plotting
%matplotlib inline

import re #regex 정규표현식
from difflib import SequenceMatcher as SM #string comparison 문자열이 얼마나 비슷한지 확인하는 모듈

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

**Preprocessing**

In [ ]:
cwur = pd.read_csv('../input/cwurData.csv')
shanghai = pd.read_csv('../input/shanghaiData.csv')
times = pd.read_csv('../input/timesData.csv')

In [ ]:
cwur = cwur.rename(columns = {'institution': 'university_name'})
shanghai = shanghai.rename(columns = {'total_score': 'score'})
times = times.rename(columns = {'total_score': 'score'})

In [ ]:
cwur = cwur[['university_name', 'score']]
shanghai = shanghai[['university_name', 'score']]
times = times[['university_name', 'score']]

#only use total score for the time being

In [ ]:
#pd.to_numeric will cast '-' to 100, therefore we gotta clean it beforhand
times = times[~(times['score'] == '-')]


In [ ]:
#convert scores to float
cwur.score = pd.to_numeric(cwur.score, errors='coerce')
shanghai.score = pd.to_numeric(shanghai.score, errors='coerce')
times.score = pd.to_numeric(shanghai.score, errors='coerce')


Creating some visualizations of score distributions to check whether a organization uses a different scala for the total score


In [ ]:
scores_cwur = [x for x in cwur['score'].values if not np.isnan(x)]
scores_shanghai = [x for x in shanghai['score'].values if not np.isnan(x)]
scores_times = [x for x in times['score'].values if not np.isnan(x)]

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(14, 5))

num_bins = 20
n, bins, patches = axes[0].hist(scores_cwur, num_bins, 
                                facecolor='green', alpha=0.5)
axes[0].set_title('Cwur')
n, bins, patches = axes[1].hist(scores_shanghai, num_bins,
                                facecolor='blue', alpha=0.5)
axes[1].set_title('Shanghai')
n, bins, patches = axes[2].hist(scores_times, num_bins, 
                                facecolor='red', alpha=0.5)
axes[2].set_title('Times')
plt.show()

In [ ]:
boxplt = plt.boxplot([scores_cwur, scores_shanghai, scores_times])
labels = plt.xticks([1, 2, 3], ['Cwur', 'Shanghai', 'Times'])

In [ ]:
Looks like Shanghai & Times do have a different scala than Cwur has thus we should standardize those

In [ ]:
cwur.score = (cwur.score - cwur.score.mean())/cwur.score.std()
shanghai.score = (shanghai.score - shanghai.score.mean())/shanghai.score.std()
times.score = (times.score - times.score.mean())/times.score.std()

In [ ]:
z_scores_cwur = [x for x in cwur['score'].values if not np.isnan(x)]
z_scores_shanghai = [x for x in shanghai['score'].values if not np.isnan(x)]
z_scores_times = [x for x in times['score'].values if not np.isnan(x)]

In [ ]:
boxplt = plt.boxplot([z_scores_cwur, z_scores_shanghai, z_scores_times])
labels = plt.xticks([1, 2, 3], ['Cwur', 'Shanghai', 'Times'])

way better, now we can try to merge those datasets



**Merging datasets**

Start by using built in SequenceMatcher


In [ ]:
def is_fuzzy_match(string1, string2, threshold = 0.9):
    similarity = SM(None, str(string1), str(string2)).ratio()
    if (similarity > threshold):
        return True
    else:
        return False

identifying the organization with the most universities present, gonna use that as a starting point



In [ ]:
(len(cwur), len(shanghai), len(times))


In [ ]:
data = cwur


In [ ]:
data = data.rename(columns = {'score': 'score_cwur'})



note: not that pretty, gonna refactor those functions later on:

more flexibility
using the country as an additional parameter
calculating stats for matching

In [ ]:
def check_for_uni_shanghai(series):
    university = series['university_name']
    for uni in shanghai['university_name'].values:
        if (is_fuzzy_match(university, uni)):
            return shanghai[shanghai['university_name'] == uni]['score'].values[0]
    #print('found no match for {u}'.format(u = university))

In [ ]:
def check_for_uni_times(series):
    university = series['university_name']
    for uni in times['university_name'].values:
        if (is_fuzzy_match(university, uni)):
            return times[times['university_name'] == uni]['score'].values[0]
    #print('found no match for {u}'.format(u = university))

In [ ]:
data['score_shanghai'] = data.apply(check_for_uni_shanghai, axis = 1)


In [ ]:
data['score_times'] = data.apply(check_for_uni_times, axis = 1)
